In [1]:
import pandas as pd

from lxml.html import parse

In [43]:
def get_table_as_df(table):
    ''' Ugly function to extract an HTML table into a DataFrame'''
    
    table_contents = []
    for e in table.getiterator():
        tr_counter = 0
        if e.tag == 'tr':
            tr_counter += 1
            tr_contents = []
            for tr_e in e.getiterator():
                if tr_e.tag == 'td':
                    td_contents = ''
                    for td_e in tr_e.getiterator():
                        if td_e.text is not None:
                            td_contents = td_contents + ' ' + td_e.text.split(' [')[0]
                    tr_contents.append(td_contents)
            table_contents.append(tr_contents)
    
    # Get the first line of the table, which are the column names
    header = table_contents.pop(0)
    df = pd.DataFrame(table_contents, columns=header)
    df.index = df[header[0]]
    df = df.drop(header[0], axis=1)
    
    return df

In [44]:
page = parse('html/CMLh5whitepaper.html')

table_dict = {}
current_key = 'foo'

for e in page.getiterator():
    if e.tag == 'h2' or e.tag == 'h1':
        print e.getchildren()[0].text
        current_key = e.getchildren()[0].text
    if e.tag == 'table':
        table_dict[current_key] = get_table_as_df(e)

None
None
None
None
1 Background and Motivation
2 CML Terminology
3 CMLh5 data format structure
3.1 Example structure of a CMLh5 file
None
3.2 Specifications of the arrays in the channel subgroup
3.3 Dimension scales
4 CMLh5 metadata conventions
4.1 Metadata at the root level
4.2 Metadata at the CML level
4.3 Metadata at the channel level
4.4 Metadata at the array level
5 Missing data conventions


In [48]:
heading_names_to_file_names = {
    'data_def_channel_level.csv' : '3.2 Specifications of the arrays in the channel subgroup',
    'metadata_def_root_level.csv': '4.1 Metadata at the root level',
    'metadata_def_cml_level.csv': '4.2 Metadata at the CML level',
    'metadata_def_channel_level.csv': '4.3 Metadata at the channel level',
    'metadata_def_array_level.csv': '4.4 Metadata at the array level',
    'missing_data_convetions.csv': '5 Missing data conventions'}

for file_name, heading_name in heading_names_to_file_names.iteritems():
    print 'writing ' + file_name
    table_dict[heading_name].to_csv(file_name,  encoding='utf-8')

writing metadata_def_array_level.csv
writing metadata_def_cml_level.csv
writing metadata_def_channel_level.csv
writing data_def_channel_level.csv
writing metadata_def_root_level.csv
writing missing_data_convetions.csv
